In [5]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

In [16]:
def asset_growth(ticker):
    """Asset growth is defined as the YoY % change in quarterly total assets.
    Higher is worse, so multiply by -1"""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    df['Asset_growth'] = df.groupby('Quarter')['Total Assets'].pct_change()*-1
    return df[['Calendar_Year', 'Quarter','Total Assets','Asset_growth']]

In [151]:
asset_growth('AAPL').tail()

,Calendar_Year,Quarter,Total Assets,Asset_growth
4,2021,Q2,"337,158.00",-0.05
3,2021,Q3,"329,840.00",-0.04
2,2021,Q4,"351,002.00",-0.08
1,2022,Q1,"381,191.00",-0.08
0,2022,Q2,"350,662.00",-0.04


In [90]:
def asset_turnover_change(ticker):
    """YoY % change in the ratio of Sales to Total Assets. Higher is better"""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    
    def divide_two_cols(df):
        df['Asset_Turnover'] = df['Revenue'] / (df['Total Assets'])
        return df
        
    df = divide_two_cols(df)
    df['Asset_Turnover_chg'] = df.groupby('Quarter')['Asset_Turnover'].pct_change()
    
    df['Asset_growth'] = df.groupby('Quarter')['Total Assets'].pct_change()*-1

    return df

In [91]:
df_AAPL = asset_turnover_change('AAPL')
df_AAPL.tail()

,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,Revenue,Gross Profit,...,PE,PS,PB,Price To FCF,PEG,EPS,Ticker,Asset_Turnover,Asset_Turnover_chg,Asset_growth
4,2021-03-27,2021,Q2,"2,030,688.81",1.57,0.79,5.35,1.41,"89,584.00","38,079.00",...,21.48,22.67,29.35,93.53,-1.26,1.41,AAPL,0.27,0.46,-0.05
3,2021-06-26,2021,Q3,"2,213,535.58",1.65,0.81,4.90,1.31,"81,434.00","35,255.00",...,25.45,27.18,34.44,116.50,-3.59,1.31,AAPL,0.25,0.31,-0.04
2,2021-09-25,2021,Q4,"2,422,287.79",1.73,0.82,5.06,1.25,"83,360.00","35,174.00",...,29.47,29.06,38.39,142.68,-6.43,1.25,AAPL,0.24,0.19,-0.08
1,2021-12-25,2022,Q1,"2,955,919.62",1.48,0.81,7.56,2.11,"123,945.00","54,243.00",...,21.34,23.85,41.09,66.93,0.31,2.11,AAPL,0.33,0.03,-0.08
0,2022-03-26,2022,Q2,"2,863,962.04",1.53,0.81,5.93,1.53,"97,278.00","42,559.00",...,28.63,29.44,42.49,111.65,-1.04,1.53,AAPL,0.28,0.04,-0.04


In [146]:
def cap_spend_growth(ticker):
    """YoY % change of: (Latest 4 quarters of cap ex + R&D expense)/(latest 4 quarters of sales)*-1"""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    df['TTM_expenses'] = (df['R&D Expenses'].rolling(4).sum()+df['CAPEX'].rolling(4).sum()*-1)
    df['TTM_sales'] = df['Revenue'].rolling(4).sum()
    df['cap_spend_%_of_rev'] = df['TTM_expenses']/df['TTM_sales']
    df['cap_spend_growth'] = df.groupby('Quarter')['cap_spend_%_of_rev'].pct_change()
    return df
    

In [147]:
cap_spend_growth('AAPL').tail()

,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,Revenue,Gross Profit,...,PS,PB,Price To FCF,PEG,EPS,Ticker,TTM_expenses,TTM_sales,cap_spend_%_of_rev,cap_spend_growth
4,2021-03-27,2021,Q2,"2,030,688.81",1.57,0.79,5.35,1.41,"89,584.00","38,079.00",...,22.67,29.35,93.53,-1.26,1.41,AAPL,"29,279.00","325,406.00",0.09,-0.08
3,2021-06-26,2021,Q3,"2,213,535.58",1.65,0.81,4.90,1.31,"81,434.00","35,255.00",...,27.18,34.44,116.50,-3.59,1.31,AAPL,"30,766.00","347,155.00",0.09,-0.07
2,2021-09-25,2021,Q4,"2,422,287.79",1.73,0.82,5.06,1.25,"83,360.00","35,174.00",...,29.06,38.39,142.68,-6.43,1.25,AAPL,"32,999.00","365,817.00",0.09,-0.05
1,2021-12-25,2022,Q1,"2,955,919.62",1.48,0.81,7.56,2.11,"123,945.00","54,243.00",...,23.85,41.09,66.93,0.31,2.11,AAPL,"33,445.00","378,323.00",0.09,-0.08
0,2022-03-26,2022,Q2,"2,863,962.04",1.53,0.81,5.93,1.53,"97,278.00","42,559.00",...,29.44,42.49,111.65,-1.04,1.53,AAPL,"34,815.00","386,017.00",0.09,0.00


In [163]:
def cap_spend_level(ticker):
    """(latest 4 quarters of cap ex + R&D expense)/latest 4 qtrs of sales*-1"""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    df['TTM_expenses'] = (df['R&D Expenses'].rolling(4).sum() +df['CAPEX'].rolling(4).sum()*-1)
    df['TTM_sales'] = df['Revenue'].rolling(4).sum()
    df['cap_spend_%_of_rev'] = df['TTM_expenses']/df['TTM_sales']
    return df[::-1]

In [164]:
cap_spend_level('AAPL').head(10)

,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,Revenue,Gross Profit,...,PE,PS,PB,Price To FCF,PEG,EPS,Ticker,TTM_expenses,TTM_sales,cap_spend_%_of_rev
0,2022-03-26,2022,Q2,"2,863,962.04",1.53,0.81,5.93,1.53,"97,278.00","42,559.00",...,28.63,29.44,42.49,111.65,-1.04,1.53,AAPL,"34,815.00","386,017.00",0.09
1,2021-12-25,2022,Q1,"2,955,919.62",1.48,0.81,7.56,2.11,"123,945.00","54,243.00",...,21.34,23.85,41.09,66.93,0.31,2.11,AAPL,"33,445.00","378,323.00",0.09
2,2021-09-25,2021,Q4,"2,422,287.79",1.73,0.82,5.06,1.25,"83,360.00","35,174.00",...,29.47,29.06,38.39,142.68,-6.43,1.25,AAPL,"32,999.00","365,817.00",0.09
3,2021-06-26,2021,Q3,"2,213,535.58",1.65,0.81,4.90,1.31,"81,434.00","35,255.00",...,25.45,27.18,34.44,116.50,-3.59,1.31,AAPL,"30,766.00","347,155.00",0.09
4,2021-03-27,2021,Q2,"2,030,688.81",1.57,0.79,5.35,1.41,"89,584.00","38,079.00",...,21.48,22.67,29.35,93.53,-1.26,1.41,AAPL,"29,279.00","325,406.00",0.09
5,2020-12-26,2021,Q1,"2,314,861.46",1.50,0.81,6.58,1.70,"111,439.00","44,328.00",...,20.13,20.77,34.96,65.65,0.16,1.70,AAPL,"28,166.00","294,135.00",0.10
6,2020-09-26,2020,Q4,"1,915,229.78",1.51,0.80,3.79,0.74,"64,698.00","24,689.00",...,37.78,29.60,29.31,101.92,2.82,0.74,AAPL,"26,061.00","274,515.00",0.09
7,2020-06-27,2020,Q3,"1,525,055.21",1.30,0.77,3.46,0.65,"59,685.00","22,680.00",...,33.88,25.55,21.10,103.70,29.14,0.65,AAPL,"26,186.00","273,857.00",0.10
8,2020-03-28,2020,Q2,"1,080,171.45",1.14,0.76,3.34,0.64,"58,313.00","22,370.00",...,24.01,18.52,13.77,94.27,-0.49,0.65,AAPL,"26,120.00","267,981.00",0.10
9,2019-12-28,2020,Q1,"1,279,478.54",1.04,0.74,5.20,1.26,"91,819.00","35,217.00",...,14.39,13.93,14.29,45.04,0.22,1.26,AAPL,"26,013.00","267,683.00",0.10


In [178]:
def working_capital_accruals(ticker):
    """Trailing 4 quarters: 
    chg in receivables + chg in inventories + chg in accts payable + chg in tax accruals + change in misc. assets and liabilites
    Scaled by avg Total Assets
    """
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    df['chgReceivables'] = df.groupby(['Quarter'])['Receivables'].diff().fillna(0)      
    df['chgInventory'] = df.groupby(['Quarter'])['Inventory'].diff().fillna(0)                      
    df['chgPayables'] = df.groupby(['Quarter'])['Payables'].diff().fillna(0)  
    df['chgTaxAcc'] = df.groupby(['Quarter'])['Tax Assets'].diff().fillna(0)
    #df['chgMisc'] = 
    df['Accruals'] = df['chgReceivables'] + df['chgInventory'] + df['chgPayables'] + df['chgTaxAcc']
    df['avgTotalAssets'] = df['Total Assets'].rolling(4).mean().fillna(0)
    df['avgAssetAccruals'] = (df['Accruals'] / df['avgTotalAssets']).fillna(0)
    return df
    

In [181]:
working_capital_accruals('AAPL').tail(3)

,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,Revenue,Gross Profit,...,PEG,EPS,Ticker,chgReceivables,chgInventory,chgPayables,chgTaxAcc,Accruals,avgTotalAssets,avgAssetAccruals
2,2021-09-25,2021,Q4,"2,422,287.79",1.73,0.82,5.06,1.25,"83,360.00","35,174.00",...,-6.43,1.25,AAPL,"14,061.00","2,519.00","12,467.00",0.00,"29,047.00","343,013.50",0.08
1,2021-12-25,2022,Q1,"2,955,919.62",1.48,0.81,7.56,2.11,"123,945.00","54,243.00",...,0.31,2.11,AAPL,"6,633.00",903.00,"10,516.00",0.00,"18,052.00","349,797.75",0.05
0,2022-03-26,2022,Q2,"2,863,962.04",1.53,0.81,5.93,1.53,"97,278.00","42,559.00",...,-1.04,1.53,AAPL,"12,364.00",241.00,"12,555.00",0.00,"25,160.00","353,173.75",0.07


## Momentum

In [26]:
def RelativeStrengthIndex(ticker, time_window):
    """RSI compares the magnitude of recent price changes across stocks
    to identify stocks as overbought or oversold.
    A high RSI (usually above 70) indicate overbought and a low RSI 
    (typically below 30) indicates oversold.
    If first computes the avg price change for a given # (often 14)
    of peior trading days with rising prices and falling prices"""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/prices/{ticker}.csv')
    df = df[::-1]
    diff = df['change'].dropna()
    
    # preserves dimensions of diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change  is equal to the positive diff, otherwise 0
    up_chg[diff > 0] = diff[diff >0]
    
    # down chg is equal to negative diff, oterhwise 0
    down_chg[diff < 0] = diff[diff < 0]
    
    up_chg_avg = up_chg.ewm(com=time_window-1, min_periods = time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1, min_periods = time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi
    
    
    

In [33]:
RelativeStrengthIndex('AAPL',14).to_frame().tail()

,change
4,70.080049
3,70.943206
2,67.725188
1,65.130935
0,63.487875


In [34]:
aapl = pd.read_csv('/Users/brendan/Desktop/Python/BoostedAI/prices/AAPL.csv')
aapl['RSI'] = RelativeStrengthIndex('AAPL',14)
aapl.head()


,Unnamed: 0,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime,Ticker,RSI
0,0,2022-07-26,152.265000,153.085000,150.800000,151.600000,151.600000,52295393.0,52295393.0,-0.665,-0.437,151.82833,"July 26, 22",-0.00437,AAPL,63.487875
1,1,2022-07-25,154.010000,155.040000,152.280000,152.950000,152.950000,53518694.0,53518694.0,-1.060,-0.688,153.42333,"July 25, 22",-0.00688,AAPL,65.130935
2,2,2022-07-22,155.390000,156.280000,153.410000,154.090000,154.090000,66626929.0,66626929.0,-1.300,-0.837,154.59333,"July 22, 22",-0.00837,AAPL,67.725188
3,3,2022-07-21,154.500000,155.570000,151.940000,155.350000,155.350000,64957826.0,64957826.0,0.850,0.550,154.28667,"July 21, 22",0.00550,AAPL,70.943206
4,4,2022-07-20,151.119995,153.720001,150.369995,153.039993,153.039993,64759500.0,64759500.0,1.920,1.271,152.37666,"July 20, 22",0.01271,AAPL,70.080049


In [40]:
def price_momentum(ticker):
    """This factor computes the total return for a given number of prior days.
        It is common to use the last 12 months except for the most recent month
    """
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/prices/{ticker}.csv')
    df = df[::-1]
    df['Momentum'] = df['changePercent'].shift(1)/df['changePercent'].shift(12)-1
    return df
    

In [48]:
def price_momentum_vol_adj(ticker):
    """The indicator normalizes the total return over the previous 12 months by dividing it by the standard deviation of these returns."""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/prices/{ticker}.csv')
    df = df[::-1]
    df['MoVolAdj'] = (df['adjClose'].shift(0)/df['adjClose'].shift(252)-1)/df['changePercent'].rolling(252).std()
    return df

In [54]:
aapl = price_momentum_vol_adj('AAPL')
len(aapl.sort_values(by=['MoVolAdj']).dropna())

1006